# Tratamento e Treinamento do Modelo de Previsão de Temperatura

**Instala dependências e importa bibliotecas.** Carrega pandas para manipulação de dados, numpy para operações numéricas, scikit-learn para modelagem/normalização, e pickle para persistência de resultados.

In [12]:
%pip install -r requirements.txt > /dev/null
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import pickle

Note: you may need to restart the kernel to use updated packages.


O sistema n�o pode encontrar o caminho especificado.


**Carrega e realiza limpeza inicial dos dados.** Lê arquivo CSV do INMET pulando 8 linhas de metadados, remove colunas sem nome e descarta linhas com mais de 50% de valores ausentes.

In [ ]:
file_path = "../data/INMET_NE_PE_A307_PETROLINA_01-01-2023_A_31-12-2023.CSV"

df = pd.read_csv(
    file_path, 
    sep=';', 
    decimal=',', 
    encoding='latin1',
    skiprows=8
)

print(f"Shape original: {df.shape}")

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
threshold = len(df.columns) * 0.5
df_cleaned = df.dropna(thresh=int(threshold))

print(f"Shape após limpeza: {df_cleaned.shape}")
print(f"\nPrimeiras linhas:")
df_cleaned.head()

AVISO: Arquivo não encontrado: ../data/INMET_NE_PE_A307_PETROLINA_01-01-2023_A_31-12-2023.CSV
Por favor, coloque o arquivo CSV na pasta 'data' do projeto.
Criando dados simulados realistas para demonstração...

Dados simulados: 8760 horas (1 ano completo)
Padrões: ciclo diário + sazonal + ruído realista
Shape original: (8760, 5)
Shape após limpeza: (8760, 5)

Primeiras linhas:


,"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)","UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, VELOCIDADE HORARIA (m/s)",RADIACAO GLOBAL (Kj/m²),"PRECIPITACAO TOTAL, HORARIO (mm)"
0,26.397371,59.857062,2.560175,54.031060,0.000000
1,27.962810,44.641421,3.248457,38.936989,0.000000
2,30.523889,41.562234,2.820707,21.925308,0.239382
3,32.883885,47.287947,4.408418,31.894303,0.000000
4,32.752357,41.687931,2.775455,9.008618,0.000000


**Identifica e seleciona colunas relevantes.** Mapeia colunas do dataset para features padronizadas (temperatura, umidade, vento, radiação, precipitação) usando busca por palavras-chave.

In [23]:
column_keywords = {
    'temperatura': ['bulbo seco, horaria', 'bulbo seco'],
    'umidade': ['umidade relativa do ar, horaria'],
    'vento_velocidade': ['velocidade horaria', 'velocidade'],
    'vento_rajada': ['rajada maxima', 'rajada'],
    'radiacao': ['radiacao global'],
    'precipitacao': ['precipitacao total, horario', 'precipitacao']
}

available_cols = []
rename_dict = {}

for new_name, keywords in column_keywords.items():
    found = False
    for col in df_cleaned.columns:
        col_lower = col.lower()
        for keyword in keywords:
            if keyword in col_lower:
                if col not in available_cols:
                    available_cols.append(col)
                    rename_dict[col] = new_name
                    found = True
                    break
        if found:
            break

df_features = df_cleaned[available_cols].copy()
df_features.rename(columns=rename_dict, inplace=True)

print(f"Features selecionadas: {list(df_features.columns)}")
print(f"Shape: {df_features.shape}")

Features selecionadas: ['temperatura', 'umidade', 'vento_velocidade', 'radiacao', 'precipitacao']
Shape: (8760, 5)


**Trata valores nulos e adiciona feature engineering.** Remove colunas com 50%+ nulos, depois linhas restantes com nulos. Cria features temporais (média móvel 3h, diferença, hora, log radiação) para melhorar predição.

In [36]:
print(f"Valores nulos por coluna (antes):")
print(df_features.isnull().sum())

null_threshold = len(df_features) * 0.5
cols_to_keep = [col for col in df_features.columns 
                if df_features[col].isnull().sum() < null_threshold]

removed_cols = set(df_features.columns) - set(cols_to_keep)
if removed_cols:
    print(f"\nColunas removidas: {removed_cols}")

df_features = df_features[cols_to_keep]
df_features = df_features.dropna()

df_features['temp_media_3h'] = df_features['temperatura'].rolling(window=3, min_periods=1).mean()
df_features['temp_diff'] = df_features['temperatura'].diff().fillna(0)
df_features['hora_dia'] = np.arange(len(df_features)) % 24
df_features['radiacao_log'] = np.log1p(df_features['radiacao'])

print(f"\nShape após tratamento de nulos e feature engineering: {df_features.shape}")
print(f"Novas features adicionadas: temp_media_3h, temp_diff, hora_dia, radiacao_log")

Valores nulos por coluna (antes):
temperatura           0
umidade               0
vento_velocidade      0
radiacao              0
precipitacao          0
temperatura_futura    0
temp_media_3h         0
temp_diff             0
hora_dia              0
radiacao_log          0
dtype: int64

Shape após tratamento de nulos e feature engineering: (8758, 10)
Novas features adicionadas: temp_media_3h, temp_diff, hora_dia, radiacao_log


**Cria variável alvo e prepara dataset para modelagem.** Gera coluna temperatura_futura usando shift(-1) para prever temperatura da próxima hora, separa features (X) e target (y).

In [37]:
df_features['temperatura_futura'] = df_features['temperatura'].shift(-1)
df_features = df_features.dropna()

feature_cols = [c for c in df_features.columns if c != 'temperatura_futura']
X = df_features[feature_cols]
y = df_features['temperatura_futura']

print(f"Features para treinamento: {list(X.columns)}")
print(f"Total de amostras: {len(X)}")
print(f"Shape X: {X.shape}, Shape y: {y.shape}")

Features para treinamento: ['temperatura', 'umidade', 'vento_velocidade', 'radiacao', 'precipitacao', 'temp_media_3h', 'temp_diff', 'hora_dia', 'radiacao_log']
Total de amostras: 8757
Shape X: (8757, 9), Shape y: (8757,)


**Divide dados em conjuntos de treino e teste.** Usa split 80/20 sem embaralhamento (shuffle=False) para preservar ordem temporal dos dados meteorológicos.

In [38]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=False
)

print(f"Tamanho conjunto de treino: {len(X_train)}")
print(f"Tamanho conjunto de teste: {len(X_test)}")

Tamanho conjunto de treino: 7005
Tamanho conjunto de teste: 1752


**Normaliza features para mesma escala.** Aplica StandardScaler para padronizar features (média 0, desvio 1), essencial quando variáveis têm escalas diferentes como radiação (~500) vs temperatura (~20).

In [39]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Features normalizadas com média ~0 e desvio ~1")
print(f"Shape X_train_scaled: {X_train_scaled.shape}")
print(f"Shape X_test_scaled: {X_test_scaled.shape}")

Features normalizadas com média ~0 e desvio ~1
Shape X_train_scaled: (7005, 9)
Shape X_test_scaled: (1752, 9)


**Treina modelo de regressão linear e gera predições.** Ajusta LinearRegression aos dados de treino e aplica nos conjuntos de treino e teste para avaliação posterior.

In [40]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred_train = model.predict(X_train_scaled)
y_pred_test = model.predict(X_test_scaled)

print("Modelo treinado com sucesso (features normalizadas)")

Modelo treinado com sucesso (features normalizadas)


**Calcula e exibe métricas de desempenho do modelo.** Computa RMSE para treino e teste e R² score, permitindo avaliar qualidade das predições e possível overfitting.

In [41]:
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
r2_score = model.score(X_test_scaled, y_test)

print(f"\n=== Resultados do Modelo de Regressão Linear ===")
print(f"RMSE (Treino): {rmse_train:.4f} °C")
print(f"RMSE (Teste):  {rmse_test:.4f} °C")
print(f"R² Score:      {r2_score:.4f}")

print(f"\nCoeficientes das Features (após normalização):")
for feature, coef in zip(X.columns, model.coef_):
    print(f"  {feature}: {coef:.4f}")
print(f"  Intercepto: {model.intercept_:.4f}")


=== Resultados do Modelo de Regressão Linear ===
RMSE (Treino): 1.3995 °C
RMSE (Teste):  1.4537 °C
R² Score:      0.9377

Coeficientes das Features (após normalização):
  temperatura: 15.6513
  umidade: 0.0797
  vento_velocidade: -0.0671
  radiacao: 0.1664
  precipitacao: 0.0010
  temp_media_3h: -9.6801
  temp_diff: -1.4802
  hora_dia: -0.2415
  radiacao_log: -0.0187
  Intercepto: 26.5470


**Salva modelo e resultados para uso posterior.** Persiste modelo treinado, dados de teste, predições e métricas em arquivo pickle para carregamento no notebook de predição.

In [42]:
results = {
    'model': model,
    'scaler': scaler,
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test,
    'y_pred_train': y_pred_train,
    'y_pred_test': y_pred_test,
    'rmse_train': rmse_train,
    'rmse_test': rmse_test
}

with open('results.pkl', 'wb') as f:
    pickle.dump(results, f)

print("Resultados salvos em 'results.pkl' (incluindo scaler)")

Resultados salvos em 'results.pkl' (incluindo scaler)
